# <center> Поиск сведений об основном ОКВЭД организации

## Постановка задачи

<U>Исходные данные:</U>
   
Имеется DataFrame "df_full" с признаками:
- "ogrn": ОГРН организации (тип int);
- "mainOkved": основной ОКВЭД организации (тип object: словарь c ключами: code - код ОКВЭД, name - наименование ОКВЭД);
- "opf": сведения об организационно-правовой форме (ОПФ) организации на основании данных ЕГРЮЛ   
(тип: object: словарь c ключами: code - код ОКВЭД, name - наименование ОКВЭД);
- "fullName": наименование организации (тип: object).

<U>Проблема:</U>  
Для ряда организаций значение признака "mainOkved" отсутствуют: представлены в виде {code: None, name: None}.  
  
<U>Постановка задачи:</U>  
Заполнить пропущенные значения признака "mainOkved".

## Анализ исходных данных

In [1]:
# Сбор данных

import pandas as pd

df = pd.read_csv("data/search_mainOkved.csv")
df.head(3)


,ogrn,fullName,mainOkved,opf
0,1101600000480,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""ИНФОРМАЦИОННО-КОНС...",74,"{'code': '96', 'name': 'Некоммерческое партнер..."
1,1101300000669,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""МЕЖРЕГИОНАЛЬНЫЙ НА...",75,"{'code': '96', 'name': 'Некоммерческое партнер..."
2,1087799022046,АССОЦИАЦИЯ ОРГАНИЗАЦИЙ СИСТЕМЫ МПС,None,"{'code': '93', 'name': 'Ассоциация (союз)', 'v..."


In [2]:
# Обработка признака opf: выделение кода из значения признака

# Функция для извлечения значений 'code' из словаря признака opf
def code_dict(df_dict):
    #аргумент df_dict: значение признака, представляет собой словарь; 
    #                             у каждого словаря есть ключи: code, name.    
    #return: код признака    
    df_dict = df_dict.replace("'", '') #Удаление апострофов из значений в столбце
    df_dict = df_dict[7:] #Удаление первых символов
    df_dict = df_dict.split(',')[0] #Удаление всех символов после запятой включительно
    return df_dict

# Обработка opf (расшифровка наименования ОПФ): выводим только код
df['opf'] = df['opf'].apply(lambda x: code_dict(x))

df[['ogrn','fullName','mainOkved','opf']].head(3)

,ogrn,fullName,mainOkved,opf
0,1101600000480,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""ИНФОРМАЦИОННО-КОНС...",74,96
1,1101300000669,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""МЕЖРЕГИОНАЛЬНЫЙ НА...",75,96
2,1087799022046,АССОЦИАЦИЯ ОРГАНИЗАЦИЙ СИСТЕМЫ МПС,None,93


In [3]:
#Соберем аналитические данные

# данные без пропущенных значений основнокго ОКВЭД
df_mainOkved = df[df['mainOkved'] != 'None'][['ogrn','fullName','mainOkved','opf']] 
 # данные только с пропущенными значениями основнокго ОКВЭД
df_none_mainOkved = df[df['mainOkved'] == 'None'][['ogrn','fullName','mainOkved','opf']]
# данные без пропущенных значений основнокго ОКВЭД и кода opf
df_mainOkved_statistic = df[(df['mainOkved'] != 'None')&(df['opf'] != 'None')][['ogrn','fullName','mainOkved','opf']]

print(f'Количество данных без пропущенных значений основнокго ОКВЭД: {df_mainOkved.shape}')
print(f'Количество данных только с пропущенными значениями основнокго ОКВЭД: {df_none_mainOkved.shape}')
print(f'Количество данных без пропущенных значений основнокго ОКВЭД и кода opf: {df_mainOkved_statistic.shape}')

Количество данных без пропущенных значений основнокго ОКВЭД: (549040, 4)
Количество данных только с пропущенными значениями основнокго ОКВЭД: (203032, 4)
Количество данных без пропущенных значений основнокго ОКВЭД и кода opf: (522647, 4)


## Гипотиза по поиску пропущенных значений основного ОКВЭД

Предположим, что основной ОКВЭД связан с:
* кодом ОПФ;
* названием организации.

Проверку связи ОКВЭД с кодом ОПФ протестируем по критерию "КРИТЕРИЙ ХИ-КВАДРАТ".

In [4]:
import scipy.stats as stats

H0 = 'Признаки "mainOkved" и "opf" независимы'
Ha = 'Между признаками "mainOkved" и "opf" существует статистически значимая связь'

# задаём уровень значимости
alpha = 0.3 

# вычисляем таблицу сопряжённости
table = pd.crosstab(df_mainOkved_statistic['mainOkved'], df_mainOkved_statistic['opf'])

# проводим тест
_, p, _, _ = stats.chi2_contingency(table)

print('p-value = {:.2f}'.format(p))

if p > alpha:
	print(H0)
else:
	print(Ha)


p-value = 0.00
Между признаками "mainOkved" и "opf" существует статистически значимая связь


Для работы с названиями организаций возьмем метод векторизации текста TfidfVectorizer, основанный на теории TF-IDF (Term Frequency-Inverse Document Frequency).         
Напишем функцию, на выходе которого по данным матрицы TF-IDF,  а также вычислениям косинусного сходства между поисковым запросом и всеми названиями организации, будет код ОКВЭД.  
С учетом предыдущего теста, который показал связь признаков "mainOkved" и "opf", включим поиск ОКВЭД в код ОПФ.      
   
Для проверки функции из данных без пропущенных значений основного ОКВЭД и кода opf выделим 10 случайных подмножеств и проверим её качество.

In [5]:
#Функции выбора главного ОКВЭД анализируемой организации

#функции для преобразования текстовых данных в числовые признаки, которые можно использовать для обучения моделей машинного обучения.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#модуль для подсчета и хранения элементов в итерируемом объекте
import collections

def mainOkved_tfidf_matrix_test(opf_kod, train_arg):
    #Значение функции (return) - экземпляр TF-IDF векторизатора, 
    #                            матрицу TF-IDF, 
    #                            перечень наименование организации, с которыми будем сравнивать
    #Аргументы - opf_kod - код opf, 
    #            train_arg - тренировочные данные.
    
    # Преобразование столбца fullName в список строк
    # Если организаций, имеющих соотвестствующий код ОПФ, больше 6, 
    # то ОКВЭД будем искать из организаций имеющих соотвестствующий код ОПФ.
    # Если организаций, имеющих соотвестствующий код ОПФ, меньше 6,
    # то ОКВЭД будем искать среди всех организаций.
    # Количество 6 определно опытным путем.
    if opf_kod != "None":
        if train_arg[train_arg['opf'] == opf_kod]['fullName'].count() > 6:
            full_names = train_arg[train_arg['opf'] == opf_kod]['fullName'].tolist()
        else:
            full_names = train_arg['fullName'].tolist()
    else:
        full_names = train_arg['fullName'].tolist()

    # Создание экземпляра TF-IDF векторизатора
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))

    # Преобразование полного списка fullName в матрицу TF-IDF
    tfidf_matrix = vectorizer.fit_transform(full_names)
    
    return vectorizer, tfidf_matrix, full_names


def mainOkved_org_test(org_name, vectorizer, tfidf_matrix, full_names, train_arg):   
    #Значение функции (return) - код главного ОКВЭД анализируемой организации
    #Аргумент - org_name - наименование анализируемой организации,
    #           vectorizer - экземпляр TF-IDF векторизатора,
    #           tfidf_matrix - матрица TF-IDF,
    #           full_names - перечень наименование организации, с которыми будем сравнивать,
    #           train_arg - тренировочные данные.

    #--------------------------------------------------------------------
    #получения списка организаций, которые схожи по имени с анализируемым
    
    # Преобразование поискового запроса в матрицу TF-IDF
    query_tfidf = vectorizer.transform([org_name])

    # Вычисление косинусного сходства между поисковым запросом и всеми fullName
    similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    # Получение индексов пяти наиболее похожих fullName    
    top_indexes = similarities.argsort()[:-6:-1]

    # Получение пяти наиболее похожих fullName
    top_full_names = [full_names[index] for index in top_indexes]
    
    #--------------------------------------------------------------------
    #выбор главного ОКВЭД анализируемой организации
    
    # Создаем пустой список для хранения значений 'mainOkved'
    list_okved = []
        
    # Перебираем каждую строку в столбце 'fullName'
    for fullname in train_arg['fullName']:
        # Проверяем, есть ли точное соответствие с любым элементом в 'top_full_names'
        if any(fullname == name for name in top_full_names):
            # Если есть соответствие, добавляем значение 'mainOkved' в список
            list_okved.append(train_arg.loc[train_arg['fullName'] == fullname, 'mainOkved'].values[0])

    # Используем Counter для подсчета количества вхождений каждого значения 'mainOkved' в 'list_okved'
    okved_counts = collections.Counter(list_okved)

    # Получаем наиболее часто встречающееся значение 'mainOkved'
    most_common_okved = okved_counts.most_common(1)[0][0]

    return most_common_okved

In [6]:
# Протестируем функции

from sklearn.model_selection import train_test_split

# Цикл случайных подмножеств, с помощью которых проверим качество функций
loop_list = [1,2,3,4,5,6,7,8,9,10]
for loop_i in loop_list:
    #Формируем выборку test и train
    train_df, test_df = train_test_split(df_mainOkved_statistic, test_size=0.0001)
    #Выделим список кодов ОПФ "opf_list", которые попали в тестовую выборку
    opf_list = test_df['opf'].unique().tolist() 
    opf_list.sort()
    #Создадим признак "predict_mainOkved" - предсказанное значение ОКВЭД
    test_df['predict_mainOkved'] = '0'
    for i in opf_list:
        vectorizer, tfidf_matrix, full_names = mainOkved_tfidf_matrix_test(i,train_df)    
        test_df_opf = test_df[test_df['opf'] == i][['ogrn','fullName','opf']]
        test_df_opf['predict_mainOkved'] = test_df_opf['fullName'].apply(lambda x: mainOkved_org_test(x, vectorizer, tfidf_matrix, full_names, train_df))
        # Объединяем DataFrame'ы по столбцу 'opf'
        merged_df = test_df.merge(test_df_opf, on=['ogrn','opf'], how='left')
        # Заполняем пропущенные значения в столбце 'mainOkved' значениями из 'mainOkved' в merged_df    
        for j in merged_df[merged_df['opf'] == i]['ogrn']:
            test_df.loc[test_df['ogrn'] == j, 'predict_mainOkved'] = merged_df.loc[merged_df['ogrn'] == j, 'predict_mainOkved_y'].values[0]
    #В тестовой выборке создаем признак 'prov': 1 - предсказано верно, 0 - предсказано не верно  
    test_df['prov'] = test_df.apply(lambda x: 1 if x['mainOkved'] == x['predict_mainOkved'] else 0, axis=1)
    #Рассчитываем процент верных предсказаний
    test_persent = round((100*test_df['prov'].sum())/test_df['prov'].count())
    print(f'{loop_i} тест. Процент верных предсказаний: {test_persent}%')
    

1 тест. Процент верных предсказаний: 66%
2 тест. Процент верных предсказаний: 70%
3 тест. Процент верных предсказаний: 72%
4 тест. Процент верных предсказаний: 77%
5 тест. Процент верных предсказаний: 64%
6 тест. Процент верных предсказаний: 70%
7 тест. Процент верных предсказаний: 72%
8 тест. Процент верных предсказаний: 62%
9 тест. Процент верных предсказаний: 64%
10 тест. Процент верных предсказаний: 70%


Вывод:  
В среднем значение процента верных предсказаний - 68,7%.  
Величина не большая, но для диплома гипотизу возьмем за основу.  
Для увеличения вероятнсти предсказания в основном файле проекта "DIPLOM_Glazyrin.ipynb" коды ОКВЭД объединим в группы по классификации ОКВЭД 2023 года.

## Заполнение пропущенных значений признака "mainOkved" 

In [ ]:
#Функции выбора главного ОКВЭД анализируемой организации.
#Функции аналогичны тестовым, но убран аргумент train_arg, вместо аргумента использется набор "df_mainOkved"

#функции для преобразования текстовых данных в числовые признаки, которые можно использовать для обучения моделей машинного обучения.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#модуль для подсчета и хранения элементов в итерируемом объекте
import collections

def mainOkved_tfidf_matrix_main(opf_kod):
    #Значение функции (return) - экземпляр TF-IDF векторизатора, 
    #                            матрицу TF-IDF, 
    #                            перечень наименование организации, с которыми будем сравнивать
    #Аргументы - opf_kod - код opf.
    
    # Преобразование столбца fullName в список строк
    # Если организаций, имеющих соотвестствующий код ОПФ, больше 6, 
    # то ОКВЭД будем искать из организаций имеющих соотвестствующий код ОПФ.
    # Если организаций, имеющих соотвестствующий код ОПФ, меньше 6,
    # то ОКВЭД будем искать среди всех организаций.
    # Количество 6 определно опытным путем.
    if opf_kod != "None":
        if df_mainOkved[df_mainOkved['opf'] == opf_kod]['fullName'].count() > 6:
            full_names = df_mainOkved[df_mainOkved['opf'] == opf_kod]['fullName'].tolist()
        else:
            full_names = df_mainOkved['fullName'].tolist()
    else:
        full_names = df_mainOkved['fullName'].tolist()

    # Создание экземпляра TF-IDF векторизатора
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))

    # Преобразование полного списка fullName в матрицу TF-IDF
    tfidf_matrix = vectorizer.fit_transform(full_names)
    
    return vectorizer, tfidf_matrix, full_names


def mainOkved_org_main(org_name, vectorizer, tfidf_matrix, full_names):   
    #Значение функции (return) - код главного ОКВЭД анализируемой организации
    #Аргумент - org_name - наименование анализируемой организации,
    #           vectorizer - экземпляр TF-IDF векторизатора,
    #           tfidf_matrix - матрица TF-IDF,
    #           full_names - перечень наименование организации, с которыми будем сравнивать,
    #           train_arg - тренировочные данные.

    #--------------------------------------------------------------------
    #получения списка организаций, которые схожи по имени с анализируемым
    
    # Преобразование поискового запроса в матрицу TF-IDF
    query_tfidf = vectorizer.transform([org_name])

    # Вычисление косинусного сходства между поисковым запросом и всеми fullName
    similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    # Получение индексов пяти наиболее похожих fullName    
    top_indexes = similarities.argsort()[:-6:-1]

    # Получение пяти наиболее похожих fullName
    top_full_names = [full_names[index] for index in top_indexes]
    
    #--------------------------------------------------------------------
    #выбор главного ОКВЭД анализируемой организации
    
    # Создаем пустой список для хранения значений 'mainOkved'
    list_okved = []
        
    # Перебираем каждую строку в столбце 'fullName'
    for fullname in df_mainOkved['fullName']:
        # Проверяем, есть ли точное соответствие с любым элементом в 'top_full_names'
        if any(fullname == name for name in top_full_names):
            # Если есть соответствие, добавляем значение 'mainOkved' в список
            list_okved.append(df_mainOkved.loc[df_mainOkved['fullName'] == fullname, 'mainOkved'].values[0])

    # Используем Counter для подсчета количества вхождений каждого значения 'mainOkved' в 'list_okved'
    okved_counts = collections.Counter(list_okved)

    # Получаем наиболее часто встречающееся значение 'mainOkved'
    most_common_okved = okved_counts.most_common(1)[0][0]

    return most_common_okved

In [ ]:
#Выделим список кодов ОПФ "opf_list", которые попали в тестовую выборку
opf_list = df_none_mainOkved['opf'].unique().tolist()
opf_list.sort()
#Создадим признак "predict_mainOkved" - предсказанное значение ОКВЭД
df_none_mainOkved['predict_mainOkved'] = '0'

#В виду того, что данных много, принято решение рабить файлы по кодам ОПФ
#Формат файла - 'data/' + count + '_opf_data.csv', где count - последовательный номер файла
#Данные файла: 'ogrn' - код ОГРН, 'predict_mainOkved' - предсказанное значение ОКВЭД.
count = 0

#Цикл формирования файлов 
for i in opf_list:
    count += 1
    vectorizer, tfidf_matrix, full_names = mainOkved_tfidf_matrix_main(i)    
    df_none_mainOkved_opf = df_none_mainOkved[df_none_mainOkved['opf'] == i][['ogrn','fullName','opf']]
    df_none_mainOkved_opf['predict_mainOkved'] = df_none_mainOkved_opf['fullName'].apply(lambda x: mainOkved_org_main(x, vectorizer, tfidf_matrix, full_names))
    # Объединяем DataFrame'ы по столбцу 'opf'
    merged_df = df_none_mainOkved.merge(df_none_mainOkved_opf, on=['ogrn','opf'], how='left')
    # Заполняем пропущенные значения в столбце 'mainOkved' значениями из 'mainOkved' в merged_df    
    for j in merged_df[merged_df['opf'] == i]['ogrn']:
        df_none_mainOkved.loc[df_none_mainOkved['ogrn'] == j, 'predict_mainOkved'] = merged_df.loc[merged_df['ogrn'] == j, 'predict_mainOkved_y'].values[0] 
    # Сохранение DataFrame в файл CSV
    file_df_none_mainOkved_opf = 'data/' + count + '_opf_data.csv'
    df_none_mainOkved[df_none_mainOkved['opf'] == i][['ogrn','predict_mainOkved']].to_csv(file_df_none_mainOkved_opf, index=False)

In [ ]:
#Сформируем единый файл кодов ОКВЭД, для организаций, в которых были пропущенные данные

import os

# Создаем пустой DataFrame, в который будем объединять данные
df_full_init_data = pd.DataFrame()

count_opf = df_none_mainOkved['opf'].nunique() #Количество уникальных кодов ОПФ

# Перебираем все файлы
for i in range(1, count_opf):
    # Формируем имя файла
    filename = f"data/{i}_opf_data.csv"
    
    # Проверяем, существует ли файл
    if os.path.exists(filename):
        # Загружаем файл во временный DataFrame
        temp_df = pd.read_csv(filename)
        
        # Объединяем временный DataFrame с основным DataFrame
        df_full_init_data = pd.concat([df_full_init_data, temp_df])

# Сбрасываем индексы
df_full_init_data.reset_index(drop=True, inplace=True)

# Сохранение DataFrame в файл CSV
df_full_init_data.to_csv('data/full_init_data.csv', index=False)